<a href="https://colab.research.google.com/github/jsilbergDS/UniTox/blob/main/Results/Upload_Processor_Pulmonary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install tqdm
!pip install openai

In [ ]:
import requests
import io
import PyPDF2
from bs4 import BeautifulSoup
import pandas as pd

import json
import requests
import pandas as pd
from openai import OpenAI
import time
import numpy as np
from tqdm import tqdm
import re

In [ ]:
fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/fdalabel-query-109276.csv")
fda_df = fda_df.sort_values(by=["Active Ingredient UNII(s)","SPL Effective Date (Month/Day/Year)"],ascending=[True,False])
import pandas as pd
import re
all_fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/fdalabel-query-378064.csv")
all_fda_df["SPL Effective Date (Month/Day/Year)"] = pd.to_datetime(all_fda_df["SPL Effective Date (Month/Day/Year)"])
all_fda_df["lower"] = all_fda_df["Generic/Proper Name(s)"].str.lower()
all_fda_df = all_fda_df.sort_values(by=["lower","SPL Effective Date (Month/Day/Year)"],ascending=[True,False])
all_fda_df["Active Ingredient(s)"] = all_fda_df["Active Ingredient(s)"].astype(str).dropna()
all_fda_df["Route(s) of Administration"] = all_fda_df["Route(s) of Administration"].astype(str).dropna()
all_fda_df["Active Ingredient UNII(s)"] = all_fda_df["Active Ingredient UNII(s)"].astype(str).dropna()
all_fda_df = all_fda_df[~all_fda_df['Active Ingredient(s)'].str.contains('and|;', regex=True, flags=re.IGNORECASE)]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("TOPICAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("IRRIGATIONAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("INTRADERMAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("INHALATION")]
all_fda_df = all_fda_df[~all_fda_df['Active Ingredient UNII(s)'].str.contains(";")]
all_fda_df = all_fda_df[all_fda_df["Marketing Category"].isin(["NDA","ANDA","BLA"])]
all_fda_df = all_fda_df.groupby(["lower","Marketing Category"]).first().reset_index()
all_fda_df = all_fda_df.sort_values(["lower","Marketing Category"],ascending=[True,False])
our_df = all_fda_df.groupby("lower").first().reset_index()

In [ ]:
fda_df["Direct FDALabel Link"] = fda_df["FDALabel Link"]
fda_df["lower"] = fda_df["Generic/Proper Name(s)"].str.lower()
fda_df = fda_df[["lower","Direct FDALabel Link"]]
our_df = our_df.merge(fda_df,on="lower",how="left").groupby("lower").first().reset_index()
our_df.loc[our_df["Direct FDALabel Link"].isna(),"Direct FDALabel Link"] = our_df.loc[our_df["Direct FDALabel Link"].isna(),"FDALabel Link"]

In [ ]:
drugs_list = []
drugs_merged_list = []
drugs_initial_responses = []
drugs_no_less_most_responses = []
drugs_yes_no_responses = []
answers_list = []
initial_prompts = []
no_less_most_prompts = []
yes_no_prompts = []
moiety_list = []
fda_label_list = []

for index, row in tqdm(our_df.iterrows(), total=our_df.shape[0]):
    url = row["Direct FDALabel Link"]
    print(url)
    url=url.replace("https://fdalabel.fda.gov:8443","https://nctr-crs.fda.gov")
    if "nctr-crs.fda.gov" in url:
        response = requests.get(url)
        if response.status_code != 200:
          print("Error!")
          continue
        response.raise_for_status()  # Raises an HTTPError for bad responses
        html_content = response.text

    # Parse HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

    # Extract plain text, removing all tags
        plain_text = soup.get_text(separator='\n', strip=True)
    elif "accessdata" in url:
        r = requests.get(url)
        f = io.BytesIO(r.content)

        reader = PyPDF2.PdfReader(f)
        plain_text = "\n".join([reader.pages[i].extract_text().strip() for i in range(0,len(reader.pages))])
        plain_text=plain_text.strip()

    initial_prompt = """Provide a summary of all the sentences of the drug label that discuss pulmonary toxicity risks and pulmonary reactions for this drug.
    In your summary of each sentence, clearly state whether the drug itself was associated with or caused the pulmonary risk."""


    no_less_most_prompt = """Given the above information about a drug, answer 'was this drug associated with No Pulmonary Toxicity, Less Pulmonary Toxicity, or Most Pulmonary Toxicity?'
        Now, answer with just one word: No, Less or Most"""
    yes_no_prompt = """Given the above information about a drug, answer 'was this drug associated with Pulmonary Toxicity?'
        Now, answer with just one word: Yes or No"""
    initial_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Read through the following drug label"},
        {"role": "user", "content":plain_text},
        {"role": "user", "content":initial_prompt}
        ],
        stream=False, temperature=0,max_tokens=2000
        )
    no_less_most_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":no_less_most_prompt}
        ],
        stream=False, temperature=0

        )
    yes_no_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":yes_no_prompt}
        ],
        stream=False, temperature=0

        )
    print(no_less_most_response.choices[0].message.content.strip()," ",yes_no_response.choices[0].message.content.strip())
    drugs_list.append(row["Generic/Proper Name(s)"])
    moiety_list.append(row["Active Moiety UNII(s)"])
    drugs_initial_responses.append(initial_response.choices[0].message.content.strip())
    drugs_no_less_most_responses.append(no_less_most_response.choices[0].message.content.strip())
    drugs_yes_no_responses.append(yes_no_response.choices[0].message.content.strip())
    initial_prompts.append(initial_prompt)
    no_less_most_prompts.append(no_less_most_prompt)
    yes_no_prompts.append(yes_no_prompt)
    fda_label_list.append(url)

    if index % 100 == 0:
      temp_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"urls":fda_label_list})
      temp_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Thursday_Pulm_direct_4o_pulmtoxicity_temp_comprehensive_"+str(index)+".csv")
final_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"urls":fda_label_list})
final_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Thursday_Pulm_direct_4o_pulmtoxicity_comprehensive.csv")

In [ ]:
final_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/Thursday_Pulm_direct_4o_pulmtoxicity_comprehensive.csv")
final_df["less_determination"].value_counts()